In [2]:
from rapidfuzz import process as pr
import pandas as pd
import numpy as np

### RapidFuzz

In [4]:
# #load set of DW keywords before fuzzyWuzzy into the file
uni_kw=pd.read_csv('../data/interim/out_2019-2021_keywords_before_FuzzyWuzzy.csv', names = ['ind', 'keyword'], header=0) 
#unique_keywords=uni_kw

In [5]:
print(len(uni_kw))
uni_kw.head()

32704


,ind,keyword
0,0,malware
1,1,state repression
2,2,climate consensus
3,3,Canary Islands
4,4,Angela I


In [6]:
ratio_array= pr.cdist(list(uni_kw['keyword']), list(uni_kw['keyword']), score_cutoff = 90)

In [5]:
len(ratio_array)
df_array = pd.DataFrame(ratio_array, columns = list(uni_kw['keyword']))
df_array.head(6)

,malware,state repression,climate consensus,Canary Islands,Angela I,Evan Rachel Wood,used cars,woodcarving,taboo,Grüne Woche,...,Drachen Lord,Missionaries,kiss-in,Fintech,Manuel Noriega,nuclear arms,processionary caterpillars,Deutsche Kammerphilharmonie Bremen,Maduro,self-sufficiency
0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# Count number of non zero values in each row
nb_non_zero = np.count_nonzero(np.asarray(ratio_array), axis=1) 

In [17]:
# Save indices of rows with more than 1 non-zero value
indices_correlating_rows = [i for i, el in enumerate(list(nb_non_zero)) if el>1]

[8,
 11,
 15,
 18,
 19,
 23,
 36,
 38,
 40,
 45,
 49,
 50,
 54,
 60,
 65,
 70,
 73,
 84,
 85,
 86,
 90,
 95,
 96,
 98,
 100,
 101,
 103,
 112,
 127,
 133,
 137,
 153,
 160,
 177,
 181,
 200,
 202,
 210,
 213,
 218,
 222,
 229,
 233,
 251,
 252,
 254,
 280,
 285,
 291,
 295,
 298,
 299,
 300,
 302,
 310,
 317,
 319,
 330,
 332,
 337,
 339,
 340,
 341,
 344,
 346,
 360,
 364,
 369,
 371,
 372,
 374,
 376,
 380,
 381,
 382,
 389,
 390,
 402,
 413,
 421,
 428,
 430,
 434,
 452,
 453,
 455,
 459,
 472,
 475,
 477,
 487,
 488,
 491,
 492,
 502,
 515,
 516,
 520,
 523,
 528,
 536,
 539,
 556,
 557,
 560,
 576,
 582,
 593,
 599,
 602,
 604,
 610,
 611,
 617,
 640,
 641,
 642,
 645,
 649,
 663,
 672,
 678,
 679,
 690,
 691,
 695,
 708,
 710,
 713,
 720,
 726,
 729,
 738,
 742,
 744,
 745,
 747,
 762,
 763,
 767,
 770,
 773,
 774,
 775,
 783,
 785,
 790,
 796,
 801,
 804,
 809,
 810,
 811,
 815,
 817,
 819,
 823,
 831,
 841,
 881,
 894,
 898,
 899,
 901,
 903,
 904,
 910,
 911,
 917,
 923,
 930,